**You could change the Runtime type as GPU. This will make your training faster.**

# Spark installation


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/IDS561_Project

/content/drive/MyDrive/IDS561_Project


In [3]:
!ls 

'27 Oct - Project Proposal - IDS 561 - Group4.gdoc'
 IDS_561_Group_Project_VER_2_Team_4.ipynb
 IDS_561_Group_Project_zillowestimate.ipynb
 MovieLens.csv
 MovieLens_RecommenderSystem.ipynb
 properties_2016.csv
 properties_2017.csv
 sample_submission.csv
 spark-3.1.2-bin-hadoop3.2
 spark-3.1.2-bin-hadoop3.2.tgz
 spark-3.1.2-bin-hadoop3.2.tgz.1
 spark-3.1.2-bin-hadoop3.2.tgz.2
 train_2016.csv
 train_2017.csv
 u.data
 zillow_data_dictionary.xlsx
 Zillow_train.csv


In [4]:
# Download Java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

We already have "spark-3.1.2-bin-hadoop3.2.tgz" downloaded in your target folder.

In [5]:
# Get Spark installer (check the path on spark.apache.org)
# !wget -v https://dlcdn.apache.org/spark/spark-3.1.2/spark-3.1.2-bin-hadoop3.2.tgz

In [6]:
!tar -xvf spark-3.1.2-bin-hadoop3.2.tgz

spark-3.1.2-bin-hadoop3.2/
spark-3.1.2-bin-hadoop3.2/R/
spark-3.1.2-bin-hadoop3.2/R/lib/
spark-3.1.2-bin-hadoop3.2/R/lib/sparkr.zip
spark-3.1.2-bin-hadoop3.2/R/lib/SparkR/
spark-3.1.2-bin-hadoop3.2/R/lib/SparkR/worker/
spark-3.1.2-bin-hadoop3.2/R/lib/SparkR/worker/worker.R
spark-3.1.2-bin-hadoop3.2/R/lib/SparkR/worker/daemon.R
spark-3.1.2-bin-hadoop3.2/R/lib/SparkR/tests/
spark-3.1.2-bin-hadoop3.2/R/lib/SparkR/tests/testthat/
spark-3.1.2-bin-hadoop3.2/R/lib/SparkR/tests/testthat/test_basic.R
spark-3.1.2-bin-hadoop3.2/R/lib/SparkR/profile/
spark-3.1.2-bin-hadoop3.2/R/lib/SparkR/profile/shell.R
spark-3.1.2-bin-hadoop3.2/R/lib/SparkR/profile/general.R
spark-3.1.2-bin-hadoop3.2/R/lib/SparkR/doc/
spark-3.1.2-bin-hadoop3.2/R/lib/SparkR/doc/sparkr-vignettes.html
spark-3.1.2-bin-hadoop3.2/R/lib/SparkR/doc/sparkr-vignettes.Rmd
spark-3.1.2-bin-hadoop3.2/R/lib/SparkR/doc/sparkr-vignettes.R
spark-3.1.2-bin-hadoop3.2/R/lib/SparkR/doc/index.html
spark-3.1.2-bin-hadoop3.2/R/lib/SparkR/R/
spark-3.1.2-

In [7]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

## Starting Point: SparkSession

In [8]:
!pip install -q findspark

In [9]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/drive/MyDrive/IDS561_Project/spark-3.1.2-bin-hadoop3.2"

In [10]:

import findspark
findspark.init()

from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

import pyspark
conf = pyspark.SparkConf()
sc = pyspark.SparkContext.getOrCreate(conf=conf)
from pyspark.sql import SQLContext
sqlcontext = SQLContext(sc)

# from pyspark.sql import SparkSession
# from pyspark.conf import SparkConf
# # from pyspark import SparkContext
# # conf = SparkConf().set('spark.ui.port', '10101')
# # sc = SparkContext(conf=conf)
# spark = SparkSession.builder.appName("Movie Recommender System").config(conf=SparkConf()).getOrCreate()

# spark = SparkSession.builder \
# .appName("Movie Recommender System") \
# .master('local[*]') \
# .config('spark.sql.execution.arrow.pyspark.enabled', True) \
# .config('spark.sql.session.timeZone', 'UTC') \
# .config('spark.driver.memory','32G') \
# .config('spark.ui.showConsoleProgress', True) \
# .config('spark.sql.repl.eagerEval.enabled', True) \
# .getOrCreate()

# Read the data and prepare for ML models

In [11]:
# save into csv format
import pandas as pd

read_file = pd.read_csv(r'/content/drive/MyDrive/IDS561_Project/u.data')
read_file.to_csv (r'/content/drive/MyDrive/IDS561_Project/MovieLens.csv', index=None)

data1 = spark.read.csv("/content/drive/MyDrive/IDS561_Project/MovieLens.csv", sep='\t')

In [ ]:
data1.show(5)
data1.printSchema()
data1.count()

+---+---+---+---------+
|_c0|_c1|_c2|      _c3|
+---+---+---+---------+
|196|242|  3|881250949|
|186|302|  3|891717742|
| 22|377|  1|878887116|
|244| 51|  2|880606923|
|166|346|  1|886397596|
+---+---+---+---------+
only showing top 5 rows

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)



100000

In [12]:
data=data1.select(data1.columns[0],data1.columns[1],data1.columns[2])
data.show()

+---+----+---+
|_c0| _c1|_c2|
+---+----+---+
|196| 242|  3|
|186| 302|  3|
| 22| 377|  1|
|244|  51|  2|
|166| 346|  1|
|298| 474|  4|
|115| 265|  2|
|253| 465|  5|
|305| 451|  3|
|  6|  86|  3|
| 62| 257|  2|
|286|1014|  5|
|200| 222|  5|
|210|  40|  3|
|224|  29|  3|
|303| 785|  3|
|122| 387|  5|
|194| 274|  2|
|291|1042|  4|
|234|1184|  2|
+---+----+---+
only showing top 20 rows



In [13]:
data = data.withColumnRenamed("_c0","user_id").withColumnRenamed("_c1","item_id").withColumnRenamed("_c2","rating")
data.show(5)
data.printSchema()

+-------+-------+------+
|user_id|item_id|rating|
+-------+-------+------+
|    196|    242|     3|
|    186|    302|     3|
|     22|    377|     1|
|    244|     51|     2|
|    166|    346|     1|
+-------+-------+------+
only showing top 5 rows

root
 |-- user_id: string (nullable = true)
 |-- item_id: string (nullable = true)
 |-- rating: string (nullable = true)



In [14]:
data.dtypes
from pyspark.sql.types import IntegerType
data = data.withColumn("user_id",data["user_id"].cast(IntegerType()))
data = data.withColumn("item_id",data["item_id"].cast(IntegerType()))
data = data.withColumn("rating",data["rating"].cast(IntegerType()))
data.dtypes

[('user_id', 'int'), ('item_id', 'int'), ('rating', 'int')]

# ALS - works best with sparse matrixes like the one we have

ALS is one of the low rank matrix approximation algorithms for collaborative filtering. ALS decomposes user-item matrix into two low rank matrixes: user matrix and item matrix. In collaborative filtering, users and products are described by a small set of latent factors that can be used to predict missing entries. And ALS algorithm learns these latent factors by matrix factorization.

Spark MLlib library for Machine Learning provides a Collaborative Filtering implementation by using Alternating Least Squares. 

In [15]:
#regression evaluator helps us in measuring the performance of ALS
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS, ALSModel

# for splitting the data and cross validation 
#Validation for hyper-parameter tuning. 
#trainvalidationsplit Randomly splits the input dataset into train and validation sets, and uses evaluation metric on the validation set to select the best model. 
from pyspark.ml.tuning import TrainValidationSplit, ParamGridBuilder, CrossValidator, CrossValidatorModel

(train, test) = data.randomSplit([0.8, 0.2])
train.cache()
test.cache()

DataFrame[user_id: int, item_id: int, rating: int]

# Initial RMSE

In [16]:
# Build the recommendation model using ALS on the training data
# Note we set cold start strategy to 'drop' to ensure we don't get NaN evaluation metrics
als1 = ALS(maxIter=5, regParam=0.01, userCol="user_id", itemCol="item_id", ratingCol="rating",
          coldStartStrategy="drop")
model1 = als1.fit(train)

# Evaluate the model by computing the RMSE on the test data
predictions = model1.transform(test)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                predictionCol="prediction")
rmse1 = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse1))

Root-mean-square error = 1.0631985073263832


In [17]:
# Building ALS Model

def asl_model(max_iter=2,reg_param=0.01,rank=5):

  als = ALS(maxIter=max_iter, regParam=reg_param,rank=rank, userCol='user_id', itemCol='item_id', ratingCol='rating',nonnegative= True,implicitPrefs = False,coldStartStrategy="drop")
  model = als.fit(train)

  # print('Model Created-->>>>>>>>>>')

  # Evaluate the model by computing the RMSE on the test data
  predictions = model.transform(test)
  evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                predictionCol="prediction")

  rmse = evaluator.evaluate(predictions)
  return(rmse)


# Hyper Parameter Tuning

## Tuning Max Iterations

In [ ]:
# Tuning Max Iterations

print('Max Iter=2')
print(asl_model()) # initial RMSE
print('------------------')
print('------------------')

print('Max Iter=10')
print(asl_model(10,0.01,5))
print('------------------')
print('------------------')

print('Max Iter=15')
print(asl_model(15,0.01,5))
print('------------------')
print('------------------')

print('Max Iter=20')
print(asl_model(20,0.01,5))
print('------------------')
print('------------------')

print('Max Iter=25')
print(asl_model(25,0.01,5))
print('------------------')
print('------------------')

Max Iter=2
1.13808713847888
------------------
------------------
Max Iter=10
0.950596539372614
------------------
------------------
Max Iter=15
0.9465089282484206
------------------
------------------
Max Iter=20
0.9466114182468709
------------------
------------------
Max Iter=25
0.9473237969639202
------------------
------------------


To build the ALS model explicitly we specify the columns. We set nonnegative as ‘True’, since we are looking at rating greater than 0. The model also gives an option to select implicit ratings. Since we are working with explicit ratings, set it to ‘False’.
When using simple random splits as in Spark’s CrossValidator or TrainValidationSplit, it is actually very common to encounter users and/or items in the evaluation set that are not in the training set. By default, Spark assigns NaN predictions during ALSModel.transform when a user and/or item factor is not present in the model.We set cold start strategy to ‘drop’ to ensure we don’t get NaN evaluation metrics


## Tuning Rank & Reg_params - choosing  max iter = 15(min rmse)

In [18]:
# Choosing Max Iter==15

# Cross Validation to find best rank and regularization parameter

def cross_valid_latent_factors(train,validate,best_max_iter,reg_pars,ranks):
  
  min_error = float('inf')
  train=train
  validate=validate
  best_rank = -1
  best_regularization = 0
  best_model = None
  best_max_iter=best_max_iter

  for rank in ranks:
    for regpar in reg_pars:
      
      als_mod = ALS(maxIter=best_max_iter, regParam=regpar,rank=rank,userCol="user_id", itemCol="item_id", ratingCol="rating",nonnegative= True,implicitPrefs = False,coldStartStrategy="drop")
      #print(rank)
      #print(regpar)
      mod=als_mod.fit(train)

      predictions = mod.transform(validate)
      evaluator = RegressionEvaluator(metricName="rmse",labelCol="rating",predictionCol="prediction")
      rmse = evaluator.evaluate(predictions)
                              
      print('{} latent factors and regularization = {}: ''validation RMSE is {}'.format(rank, regpar, rmse))

      if rmse < min_error:
        min_error = rmse
        best_rank = rank
        best_regularization = regpar
        best_model = mod

  
  print('\nThe best model has {} latent factors and ''regularization = {}'.format(best_rank, best_regularization))
  
  return best_model


Finding best parameters - train and validation data

In [18]:
# (training, validate) = train.randomSplit([0.80, 0.20])
# import numpy as np

# best_max_iter=15
# ranks=np.arange(1,20,2)
# reg_pars=np.arange(0.05,.5,0.05)
# best_model=cross_valid_latent_factors(training,validate,best_max_iter,reg_pars,ranks)



1 latent factors and regularization = 0.05: validation RMSE is 0.9458646838505506
1 latent factors and regularization = 0.1: validation RMSE is 0.9487070815300608
1 latent factors and regularization = 0.15000000000000002: validation RMSE is 0.953554905885189
1 latent factors and regularization = 0.2: validation RMSE is 0.9611689074040866
1 latent factors and regularization = 0.25: validation RMSE is 0.9717259675285659
1 latent factors and regularization = 0.3: validation RMSE is 0.9850624093829454
1 latent factors and regularization = 0.35000000000000003: validation RMSE is 1.0009143875536273
1 latent factors and regularization = 0.4: validation RMSE is 1.019051527578419
1 latent factors and regularization = 0.45: validation RMSE is 1.039300135260476
3 latent factors and regularization = 0.05: validation RMSE is 0.9319315179061519
3 latent factors and regularization = 0.1: validation RMSE is 0.9242128788274686
3 latent factors and regularization = 0.15000000000000002: validation RMSE i

## Train the model with the best parameters

In [20]:
best_model=ALS(maxIter=15, regParam=0.15,rank=15,userCol="user_id", itemCol="item_id", ratingCol="rating",nonnegative= True,implicitPrefs = False,coldStartStrategy="drop")
best_model=best_model.fit(train)

# RMSE after hyperparameter tuning

In [22]:
predictions2 = best_model.transform(test)
evaluator2 = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                predictionCol="prediction")

rmse2 = evaluator2.evaluate(predictions2)

print("rmse initial :" , rmse1)
print("rmse post hyperparameter tuning :" , rmse2)

rmse initial : 1.0631985073263832
rmse post hyperparameter tuning : 0.9238761162250243


# Using 10 fold cross validation

using best parameters defined from hyperparameter tuning - we now do 10 fold cross validation. The goal of cross-validation is to estimate the expected level of fit of a model to a data set that is independent of the data that were used to train the model.

In [24]:
def kfoldALS(data, k, userCol='user_id', itemCol='item_id', ratingCol='rating', metricName='rmse'):
    evaluations = []
    weights = [1.0] * k
    splits = data.randomSplit(weights)
    for i in range(0, k):  
        testingSet = splits[i]
        trainingSet = spark.createDataFrame(sc.emptyRDD(), data.schema)
        for j in range(0, k):
            if i == j:
                continue
            else:
                trainingSet = trainingSet.union(splits[j])
        # als = ALS(userCol=userCol, itemCol=itemCol, ratingCol=ratingCol)
        als = ALS(maxIter=15, regParam=0.15,rank=15,userCol="user_id", itemCol="item_id", ratingCol="rating",nonnegative= True,implicitPrefs = False,coldStartStrategy="drop")
        model = als.fit(trainingSet)
        predictions = model.transform(testingSet)
        evaluator = RegressionEvaluator(metricName=metricName, labelCol='rating', predictionCol='prediction')
        evaluation = evaluator.evaluate(predictions.na.drop())
        print('Loop {}: {} = {}'.format(i + 1, metricName, evaluation))
        evaluations.append(evaluation)
    return sum(evaluations) / float(len(evaluations))

In [26]:
print('RMSE with k=10 = {}'.format(kfoldALS(data, k=10)))

Loop 1: rmse = 0.9092964170044131
Loop 2: rmse = 0.918736575191373
Loop 3: rmse = 0.92076777933965
Loop 4: rmse = 0.9139803316662796
Loop 5: rmse = 0.9098648509900733
Loop 6: rmse = 0.9254858274885077
Loop 7: rmse = 0.9196514372718952
Loop 8: rmse = 0.9199557567756307
Loop 9: rmse = 0.9137507589268424
Loop 10: rmse = 0.9183185313364394
RMSE with k=10 = 0.9169808265991103


**We have seen that using CV the rmse further reduces and this proves our model was best trained**

# Product recommentation for users

In [32]:
# Get top 10 product recommendations for all users
product_recommendation = best_model.recommendForAllUsers(10)
product_recommendation.show()
import pandas as pd
product_recommendation = product_recommendation.toPandas()

+-------+--------------------+
|user_id|     recommendations|
+-------+--------------------+
|    471|[{1427, 4.445892}...|
|    463|[{1233, 4.04156},...|
|    833|[{1597, 4.18277},...|
|    496|[{1388, 4.3781123...|
|    148|[{1642, 4.75437},...|
|    540|[{1642, 4.782356}...|
|    392|[{1463, 5.1461515...|
|    243|[{1367, 4.760416}...|
|    623|[{1347, 4.6259127...|
|    737|[{1367, 4.9752774...|
|    897|[{1467, 4.790213}...|
|    858|[{1347, 4.3194323...|
|     31|[{1500, 4.9118166...|
|    516|[{1642, 5.1860557...|
|    580|[{899, 4.759349},...|
|    251|[{1467, 4.93089},...|
|    451|[{313, 3.783609},...|
|     85|[{1463, 4.386476}...|
|    137|[{1427, 5.506092}...|
|    808|[{1500, 5.39749},...|
+-------+--------------------+
only showing top 20 rows



In [33]:
users = []
recommendations = []

# For the entire data iterate 
for i in range(len(product_recommendation)):
  # First we add user_id to the list
  users.append(product_recommendation.iloc[i,0])
  user_recs = "" 
  # Fetching only the item ID's from the recommendations
  for item in product_recommendation.iloc[i,1]:
    user_recs = user_recs + ", " + str(item.asDict()["item_id"])

  # Append the itemID's to a list
  recommendations.append(user_recs[2:])

#Convert to dataframe
recommendations_df = pd.DataFrame(data = zip(users, recommendations), columns=["user_id", "item_id"])


In [34]:
#Check the users and the top 10 movie recommendations for the first 10 users
recommendations_df.head(10)

,user_id,item_id
0,471,"1427, 1169, 867, 814, 362, 1233, 1347, 1016, 1..."
1,463,"1233, 1642, 113, 1449, 753, 408, 1251, 1377, 1..."
2,833,"1597, 1536, 1367, 1499, 1449, 641, 56, 127, 16..."
3,496,"1388, 1467, 1367, 1642, 1500, 1589, 850, 169, ..."
4,148,"1642, 169, 1449, 408, 1062, 114, 1467, 1500, 1..."
5,540,"1642, 1449, 1467, 1347, 1398, 1122, 1367, 169,..."
6,392,"1463, 1449, 1367, 1643, 1642, 1467, 1347, 1500..."
7,243,"1367, 1449, 1642, 1398, 1347, 1122, 1104, 408,..."
8,623,"1347, 1643, 1449, 50, 318, 1427, 1122, 1463, 1..."
9,737,"1367, 1104, 1449, 1536, 408, 1642, 1499, 169, ..."


In [53]:
recommendations_df.count()
type(recommendations_df)

pandas.core.frame.DataFrame

In [35]:
# exporting the output

import os
os.chdir('/content/drive/MyDrive/IDS561_Project')

recommendations_df.to_csv('HW4_movie_recommendations.csv',sep='\t',index=False,header=False)